In [ ]:
import torch
import torch.nn.functional as F
import numpy
import time
import os
CURRENT_PATH = os.getcwd()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from olympics_engine.scenario import wrestling
from olympics_engine.generator import create_scenario
gamemap =  create_scenario('wrestling')        #load map config
game = wrestling(gamemap)
obs = game.reset()

import matplotlib.pyplot as plt
plt.imshow(obs[0]['agent_obs']) 

print(obs[1]['agent_obs'])

In [ ]:
# 加载专家策略
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.Conv = torch.nn.Conv2d(in_channels=1, out_channels=1 ,kernel_size=3, padding=1)
        self.BN = torch.nn.BatchNorm2d(num_features=1)
        self.Pool = torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

        self.NN1 = torch.nn.Linear(1600, 400)
        self.NN2 = torch.nn.Linear(400, 128)
        self.NN3 = torch.nn.Linear(128, 64)
        self.NN4 = torch.nn.Linear(64, 2)


    def forward(self, input):
        x = self.Conv(input)
        x = self.BN(x)
        x = F.relu(x)
        x = self.Pool(x)
        x = F.dropout(x, p=0.2)
        x = torch.flatten(input, start_dim=1)

        x = self.NN1(x)
        x = F.relu(x)
        x = self.NN2(x)
        x = F.relu(x)
        x = self.NN3(x)
        x = F.relu(x)
        x = self.NN4(x)

        x[:, 0] = torch.tanh(x[:, 0])*150 + 50
        x[:, 1] = torch.tanh(x[:, 1])*30

        return x

expert = CNN().to(device)
expert.load_state_dict(torch.load('./expert.pt'))
expert.eval()


In [ ]:
# 设置训练网络
class agent(torch.nn.Module):
    def __init__(self):
        super(agent, self).__init__()
        self.Conv = torch.nn.Conv2d(in_channels=1, out_channels=1 ,kernel_size=3, padding=1)
        self.BN = torch.nn.BatchNorm2d(num_features=1)
        self.Pool = torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

        self.NN1 = torch.nn.Linear(1600, 800)
        self.NN2 = torch.nn.Linear(800, 128)
        self.NN3 = torch.nn.Linear(128, 64)
        self.NN4 = torch.nn.Linear(64, 2)


    def forward(self, input):
        x = self.Conv(input)
        x = self.BN(x)
        x = F.relu(x)
        x = self.Pool(x)
        x = F.dropout(x, p=0.2)
        x = torch.flatten(input, start_dim=1)

        x = self.NN1(x)
        x = F.relu(x)
        x = self.NN2(x)
        x = F.relu(x)
        x = self.NN3(x)
        x = F.relu(x)
        x = self.NN4(x)

        x[:, 0] = torch.tanh(x[:, 0])*150 + 50
        x[:, 1] = torch.tanh(x[:, 1])*30

        return x

model = agent().to(device)


In [ ]:
import random
import pygame


print('action dim 0 range = ', game.action_f)
print('action dim 1 range = ', game.action_theta)
our_team = 0        #we control team_0
obs = game.reset()

done = False
RENDER = True

if RENDER:
    game.render()

while not done:
    #学习的策略
    #action_team_0 = [random.uniform(-100,200), random.uniform(-30, 30)]     #for now we use random actions for both teams
    #action_team_0 = [26, 0]
    agent_obs = torch.tensor([[obs[0]['agent_obs']]], dtype=torch.float32)
    agent_obs = agent_obs.to(device)
    agent_action = model(agent_obs)[0].cpu()
    action_team_0 = [agent_action[0].item(), agent_action[1].item()]


    #专家策略给的行动
    expert_obs = torch.tensor([[obs[1]['agent_obs']]], dtype=torch.float32)
    expert_obs = expert_obs.to(device)
    expert_action = expert(expert_obs)[0].cpu()
    action_team_expert = [expert_action[0].item(), expert_action[1].item()]

    input_action = [action_team_0, action_team_expert]

    next_obs, reward, done, _ = game.step(input_action)
    if RENDER:
        time.sleep(0.05)
        game.render()
    
    obs = next_obs
print('final reward = ', reward)
pygame.display.quit()
#pygame.quit()